# NFL Data Extraction — 2025 Season
This notebook extracts NFL team, player, and schedule data using the [`nflreadpy`](https://github.com/nflverse/nflreadpy) package.  
The data is loaded into a SQL Server database for transformation and analysis in subsequent steps.

**Steps:**
1. Install required packages  
2. Import libraries  
3. Define target columns and data types  
4. Load data from NFLReadPy  
5. Store in SQL Server

In [ ]:
#Installing the NFL Data package (data from: https://github.com/nflverse/nflreadpy)
%pip install nflreadpy
#Installing the connector to SQL Server
%pip install pyodbc

#importing packages and libraries needed
import nflreadpy as nfl #nfl data retrieval for analysis and database creation (from: https://github.com/nflverse/nflreadpy)
from sqlalchemy import create_engine, VARCHAR, Integer, text, Date, Time, Float #SQL database interaction
import pandas as pd #Data manipluation
import urllib # Used for engine connection (simplifying connection string)
import pyodbc #For conneection to SQL Server

params = urllib.parse.quote_plus(
            "DRIVER={ODBC Driver 17 for SQL Server};"
            "SERVER=LYNN\SQLEXPRESS01;"
            "DATABASE=nfl_portfolio_project;"
            "Trusted_Connection=yes"
        )
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

In [5]:
#---------------------------------------------------------------------------------------------------------------#
# DATA PREPARATION
    
    #The 'Dtype Mapping' is there to ensure the data has the correct Datatypes and mactches with the SQL Tables
    #The 'Taget Columns" are a way of tidying up the data, only picking the columns needed for analysis
#----------------------------------------------------------------------------------------------------------------#

#Dtype mapping for each DataFram, to ensure data matches SQL Table
team_stats_dtype_mapping = {'season' : Integer, 'week' : Integer, 'team' : VARCHAR(length=255), 'season_type' : VARCHAR(length=255), 'opponent_team' : VARCHAR(length=255), 'completions' : Integer, 'attempts' : Integer, 'passing_yards' : Integer, 'passing_tds' : Integer, 'passing_interceptions' : Integer, 'sacks_suffered' : Integer, 'sack_yards_lost' : Integer, 'sack_fumbles' : Integer, 'passing_air_yards' : Integer, 'passing_yards_after_catch' : Integer, 'passing_first_downs' : Integer, 'passing_2pt_conversions' : Integer, 'carries' : Integer, 'rushing_yards' : Integer, 'rushing_tds' : Integer, 'rushing_fumbles' : Integer, 'rushing_first_downs' : Integer, 'rushing_2pt_conversions' : Integer, 'receptions' : Integer, 'targets' : Integer, 'receiving_yards' : Integer, 'receiving_tds' : Integer, 'receiving_fumbles' : Integer, 'receiving_air_yards' : Integer, 'receiving_yards_after_catch' : Integer, 'receiving_first_downs' : Integer, 'receiving_2pt_conversions' : Integer, 'special_teams_tds' : Integer, 'def_tackles_solo' : Integer, 'def_tackles_with_assist' : Integer, 'def_tackle_assists' : Integer, 'def_sacks' : Float, 'def_interceptions' : Integer, 'def_pass_defended' : Integer, 'def_tds' : Integer, 'def_fumbles' : Integer, 'penalties' : Integer, 'penalty_yards' : Integer, 'timeouts' : Integer, 'punt_returns' : Integer, 'fg_made' : Integer, 'fg_att' : Integer, 'fg_missed' : Integer, 'fg_blocked' : Integer, 'fg_made_list' : VARCHAR(length=255), 'fg_missed_list' : VARCHAR(length=255), 'pat_made' : Integer, 'pat_att' : Integer, 'pat_missed' : Integer, 'pat_blocked' : Integer}
player_stats_dtype_mapping = {'player_id' : VARCHAR(length=255), 'player_name' : VARCHAR(length=255), 'player_display_name' : VARCHAR(length=255), 'position' : VARCHAR(length=255), 'position_group' : VARCHAR(length=255), 'season' : Integer, 'week' : Integer, 'season_type' : VARCHAR(length=255), 'team' : VARCHAR(length=255), 'opponent_team' : VARCHAR(length=255), 'completions' : Integer, 'attempts' : Integer, 'passing_yards' : Integer, 'passing_tds' : Integer, 'passing_interceptions' : Integer, 'sacks_suffered' : Integer, 'sack_yards_lost' : Integer, 'sack_fumbles' : Integer, 'passing_air_yards' : Integer, 'passing_yards_after_catch' : Integer, 'passing_first_downs' : Integer, 'passing_2pt_conversions' : Integer, 'carries' : Integer, 'rushing_yards' : Integer, 'rushing_tds' : Integer, 'rushing_fumbles' : Integer, 'rushing_first_downs' : Integer, 'rushing_2pt_conversions' : Integer, 'receptions' : Integer, 'targets' : Integer, 'receiving_yards' : Integer, 'receiving_tds' : Integer, 'receiving_fumbles' : Integer, 'receiving_air_yards' : Integer, 'receiving_yards_after_catch' : Integer, 'receiving_first_downs' : Integer, 'receiving_2pt_conversions' : Integer, 'special_teams_tds' : Integer, 'def_tackles_solo' : Integer, 'def_tackles_with_assist' : Integer, 'def_tackle_assists' : Integer, 'def_sacks' : Float, 'def_interceptions' : Integer, 'def_pass_defended' : Integer, 'def_tds' : Integer, 'def_fumbles' : Integer, 'penalties' : Integer, 'penalty_yards' : Integer, 'punt_returns' : Integer, 'fg_made' : Integer, 'fg_att' : Integer, 'fg_missed' : Integer, 'fg_blocked' : Integer, 'fg_made_list' : VARCHAR(length=255), 'fg_missed_list' : VARCHAR(length=255), 'pat_made' : Integer, 'pat_att' : Integer, 'pat_missed' : Integer, 'pat_blocked' : Integer, 'fantasy_points' : Float}
season_schedule_dtype_mapping = {'game_id' : VARCHAR(length=255), 'season' : Integer(), 'game_type' : VARCHAR(length=255), 'week' : Integer(), 'gameday' : Date(), 'weekday' : VARCHAR(length=255), 'gametime' : Time(), 'away_team' : VARCHAR(length=255), 'away_score' : Integer(), 'home_team' : VARCHAR(length=255), 'home_score' : Integer(), 'location' : VARCHAR(length=255), 'result' : Integer(), 'total' : Integer(), 'overtime' : Integer, 'old_game_id' : Integer(), 'gsis' : Integer(), 'away_rest' : Integer(), 'home_rest' : Integer(), 'roof' : VARCHAR(length=255), 'surface' : VARCHAR(length=255), 'away_qb_id' : VARCHAR(length=255), 'home_qb_id' : VARCHAR(length=255), 'away_coach' : VARCHAR(length=255), 'home_coach' : VARCHAR(length=255)}
player_info_dtype_mapping = {'gsis_id' : VARCHAR(255), 'display_name' : VARCHAR(255), 'first_name' : VARCHAR(255), 'last_name' : VARCHAR(255), 'birth_date' : Date, 'position_group' : VARCHAR(255), 'position' : VARCHAR(255), 'height' : Integer, 'weight' : Integer, 'college_name' : VARCHAR(255), 'college_conference' : VARCHAR(255), 'jersey_number' : Integer, 'rookie_season' : Integer, 'last_season' : Integer, 'latest_team' : VARCHAR(255), 'status' : VARCHAR(255), 'years_of_experience' : Integer, 'draft_year' : Integer, 'draft_round' : Integer, 'draft_pick' : Integer, 'draft_team' : VARCHAR(255)}


#Target Columns, we only want extracts the columns that I need
target_columns_team_stats = ["season", "week", "team", "season_type", "opponent_team", "completions", "attempts", "passing_yards", "passing_tds", "passing_interceptions", "sacks_suffered", "sack_yards_lost", "sack_fumbles", "passing_air_yards", "passing_yards_after_catch", "passing_first_downs", "passing_2pt_conversions", "carries", "rushing_yards", "rushing_tds", "rushing_fumbles", "rushing_first_downs", "rushing_2pt_conversions", "receptions", "targets", "receiving_yards", "receiving_tds", "receiving_fumbles", "receiving_air_yards", "receiving_yards_after_catch", "receiving_first_downs", "receiving_2pt_conversions", "special_teams_tds", "def_tackles_solo", "def_tackles_with_assist", "def_tackle_assists", "def_sacks", "def_interceptions", "def_pass_defended", "def_tds", "def_fumbles", "penalties", "penalty_yards", "timeouts", "punt_returns", "fg_made", "fg_att", "fg_missed", "fg_blocked", "fg_made_list", "fg_missed_list", "pat_made", "pat_att", "pat_missed", "pat_blocked"]
target_columns_player_stats = ["player_id", "player_name", "player_display_name", "position", "position_group", "season", "week", "season_type", "team", "opponent_team", "completions", "attempts", "passing_yards", "passing_tds", "passing_interceptions", "sacks_suffered", "sack_yards_lost", "sack_fumbles", "passing_air_yards", "passing_yards_after_catch", "passing_first_downs", "passing_2pt_conversions", "carries", "rushing_yards", "rushing_tds", "rushing_fumbles", "rushing_first_downs", "rushing_2pt_conversions", "receptions", "targets", "receiving_yards", "receiving_tds", "receiving_fumbles", "receiving_air_yards", "receiving_yards_after_catch", "receiving_first_downs", "receiving_2pt_conversions", "special_teams_tds", "def_tackles_solo", "def_tackles_with_assist", "def_tackle_assists", "def_sacks", "def_interceptions", "def_pass_defended", "def_tds", "def_fumbles", "penalties", "penalty_yards", "punt_returns", "fg_made", "fg_att", "fg_missed", "fg_blocked", "fg_made_list", "fg_missed_list", "pat_made", "pat_att", "pat_missed", "pat_blocked", "fantasy_points"]
target_columns_schedule = ["game_id", "season", "game_type", "week", "gameday", "weekday", "gametime", "away_team", "away_score", "home_team", "home_score", "location", "result", "total", "overtime", "old_game_id", "gsis", "away_rest", "home_rest", "roof", "surface", "away_qb_id", "home_qb_id", "away_coach", "home_coach"]
target_columns_player_info = ["gsis_id", "display_name", "first_name", "last_name", "birth_date", "position_group", "position", "height", "weight", "college_name", "college_conference", "jersey_number", "rookie_season", "last_season", "latest_team", "status", "years_of_experience", "draft_year", "draft_round", "draft_pick", "draft_team"]

In [6]:
#-----------------#
# DATA EXTRACTION
#-----------------#

#Load the latest datat into Pandas DataFrames (selecting our Target Columns)
df_team_stats = nfl.load_team_stats().to_pandas()[target_columns_team_stats] #team game or season statistics (current season)
df_player_stats = nfl.load_player_stats().to_pandas()[target_columns_player_stats] #player game or season statistics (current season)
df_season_schedule = nfl.load_schedules(2025).to_pandas()[target_columns_schedule] #game schedules and results (current season), #This data will need updating later on with, updated scores
df_player_info = nfl.load_players().to_pandas()[target_columns_player_info] # player information

#---------------------#
# DATA IMPORT TO SQL
#---------------------#

#Load data into SQL (current season)
df_team_stats.to_sql('nfl_team_stats', engine, if_exists='replace', chunksize=5000, dtype=team_stats_dtype_mapping, index=False)
df_player_stats.to_sql('nfl_player_stats', engine, if_exists='replace', chunksize=5000, dtype=player_stats_dtype_mapping, index=False)
df_season_schedule.to_sql('nfl_season_schedule', engine, if_exists='replace', chunksize=5000, dtype=season_schedule_dtype_mapping, index=False)
df_player_info.to_sql('nfl_player_info', engine, if_exists='replace', chunksize=5000, dtype=player_info_dtype_mapping, index=False)

df_team_stats.head()

,season,week,team,season_type,opponent_team,completions,attempts,passing_yards,passing_tds,passing_interceptions,...,fg_made,fg_att,fg_missed,fg_blocked,fg_made_list,fg_missed_list,pat_made,pat_att,pat_missed,pat_blocked
0,2025,1,ARI,REG,NO,21,29,163,2,0,...,2,3,0,1,42;50,None,2,2,0,0
1,2025,1,ATL,REG,TB,27,42,298,1,0,...,2,3,1,0,41;36,44,2,2,0,0
2,2025,1,BAL,REG,BUF,14,19,209,2,0,...,2,2,0,0,52;49,None,4,5,1,0
3,2025,1,BUF,REG,BAL,33,46,394,2,0,...,3,3,0,0,25;43;32,None,2,2,0,0
4,2025,1,CAR,REG,JAX,18,35,154,1,2,...,1,1,0,0,48,None,1,1,0,0


### Summary
- Extracted 2025 NFL team, player, and schedule data via `nflreadpy`
- Selected only analysis-relevant columns for each dataset
- Loaded 4 cleaned tables into SQL Server:
  - `nfl_team_stats`
  - `nfl_player_stats`
  - `nfl_season_schedule`
  - `nfl_player_info`
  
Next: Weekly data extraction & import process (Notenook 2)